In [1]:
import numpy as np
import pandas as pd

from cyclops.processors.aggregate import Aggregator
from cyclops.processors.cleaning import (
    normalize_categories,
    normalize_names,
    normalize_values,
)
from cyclops.processors.column_names import (
    ADMIT_TIMESTAMP,
    AGE,
    CARE_UNIT,
    DIAGNOSIS_CODE,
    DIAGNOSIS_TRAJECTORY,
    ENCOUNTER_ID,
    EVENT_CATEGORY,
    EVENT_NAME,
    EVENT_TIMESTAMP,
    EVENT_VALUE,
    EVENT_VALUE_UNIT,
    HOSPITAL_ID,
    SEX,
    SUBJECT_ID,
    TIMESTEP,
    YEAR,
)
from cyclops.processors.constants import (
    BINARY,
    CATEGORICAL_INDICATOR,
    FEATURE_INDICATOR_ATTR,
    FEATURE_MAPPING_ATTR,
    FEATURE_TYPE_ATTR,
    FEATURE_TYPES,
    FEATURES,
    MEAN,
    MIN_MAX,
    MISSING_CATEGORY,
    NUMERIC,
    ORDINAL,
    STANDARD,
    STRING,
    TARGETS,
)
from cyclops.processors.feature.feature import TabularFeatures, TemporalFeatures
from cyclops.processors.feature.normalization import GroupbyNormalizer
from cyclops.processors.feature.split import split_datasets
from cyclops.query import mimic
from cyclops.query import process as qp
from cyclops.utils.file import load_dataframe, save_dataframe

2022-07-13 18:01:38,875 INFO cyclops.orm     - Database setup, ready to run queries!


In [ ]:
YEARS = [2015, 2016, 2018, 2019, 2020]
MIN_YEAR = min(YEARS)
TARGET_NAME = "death"

# Querying

In [ ]:
encounters_interface = mimic.patient_encounters(
    years=YEARS, died=True, died_binarize_col="died"
)
encounters_query = encounters_interface.query
encounters_query = qp.Drop(
    ["insurance", "language", "marital_status", "hospital_expire_flag"]
)(encounters_query)
encounters_interface = mimic.get_interface(encounters_query)
encounters = encounters_interface.run()
encounters

In [ ]:
target_events = encounters[encounters["died"] == True]
target_events = target_events[[ENCOUNTER_ID, "deathtime"]]
target_events = target_events.rename({"deathtime": EVENT_TIMESTAMP}, axis=1)
target_events[EVENT_NAME] = TARGET_NAME
target_events[EVENT_VALUE] = 1
target_events.head(5)

In [ ]:
save_dataframe(encounters, "encounters.parquet")

In [ ]:
events_interface = mimic.events()
events_query = events_interface.query
events_query = qp.Drop(["warning", "itemid", "storetime"])(events_query)
events_interface = mimic.get_interface(events_query)
events = events_interface.run(limit=1000000)
events = pd.merge(
    encounters[[ENCOUNTER_ID, "anchor_year_difference"]], events, on=ENCOUNTER_ID
)


def add_offset(row):
    row[EVENT_TIMESTAMP] += pd.DateOffset(years=row["anchor_year_difference"])
    return row


events = events.apply(add_offset, axis=1)
events = events.drop("anchor_year_difference", axis=1)

# Include target as an event
events = pd.concat([events, target_events])

events

In [ ]:
save_dataframe(events, "events.parquet")

------------------------------------------------------------------------------------------------

# Processing

## Tabular

In [2]:
encounters = load_dataframe("encounters.parquet")
encounters.head(5)

2022-07-13 18:01:38,898 INFO cyclops.utils.file - Loading dataframe from encounters.parquet


,subject_id,encounter_id,admit_timestamp,discharge_timestamp,deathtime,admission_type,admission_location,discharge_location,ethnicity,edregtime,edouttime,sex,age,birth_year,dod,anchor_year_difference,died
0,19759112,20610510,2018-07-03 02:10:00,2018-07-05 12:15:00,NaT,ELECTIVE,None,HOME,UNKNOWN,NaT,NaT,M,0,2018,NaT,-140,False
1,17978591,21300810,2015-10-10 00:21:00,2015-10-12 14:10:00,NaT,ELECTIVE,None,HOME,WHITE,NaT,NaT,F,0,2015,NaT,-134,False
2,16420748,23082241,2018-10-25 22:30:00,2018-10-28 12:58:00,NaT,ELECTIVE,None,HOME,WHITE,NaT,NaT,F,0,2018,NaT,-151,False
3,15206017,25518306,2018-10-22 15:28:00,2018-10-23 19:13:00,NaT,EU OBSERVATION,CLINIC REFERRAL,None,BLACK/AFRICAN AMERICAN,2110-10-22 11:52:00,2110-10-22 17:37:00,F,76,1942,NaT,-92,None
4,14695283,23679175,2018-06-24 18:54:00,2018-06-25 20:15:00,NaT,EU OBSERVATION,PHYSICIAN REFERRAL,None,WHITE,2176-06-24 14:46:00,2176-06-25 20:15:00,F,90,1928,NaT,-158,None


In [3]:
features = [
    AGE,
    SEX,
    "admission_type",
    "admission_location",
    "discharge_location",
    "ethnicity",
]

tab_features = TabularFeatures(
    data=encounters,
    features=features,
    by=ENCOUNTER_ID,
)

In [4]:
tab_features.get_data()

/home/kmckeen/gemini/cyclops/cyclops/processors/feature/feature.py:256: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].astype(int)


,sex,discharge_location,admission_location,ethnicity,admission_type,age,encounter_id
0,1,6.0,NaN,6,3,0,20610510
1,0,6.0,NaN,7,3,0,21300810
2,0,6.0,NaN,7,3,0,23082241
3,0,NaN,1.0,2,4,76,25518306
4,0,NaN,6.0,7,4,90,23679175
...,...,...,...,...,...,...,...
224759,1,12.0,8.0,7,8,39,25481180
224760,0,6.0,2.0,7,5,42,20965895
224761,1,12.0,2.0,7,6,58,20481995
224762,1,12.0,2.0,7,6,91,21403223


In [5]:
tab_features.types

{'admission_type': 'ordinal',
 'sex': 'binary',
 'age': 'numeric',
 'discharge_location': 'ordinal',
 'admission_location': 'ordinal',
 'ethnicity': 'ordinal'}

In [6]:
tab_features.meta["admission_location"].get_mapping()

{0: 'AMBULATORY SURGERY TRANSFER',
 1: 'CLINIC REFERRAL',
 2: 'EMERGENCY ROOM',
 3: 'INFORMATION NOT AVAILABLE',
 4: 'INTERNAL TRANSFER TO OR FROM PSYCH',
 5: 'PACU',
 6: 'PHYSICIAN REFERRAL',
 7: 'PROCEDURE SITE',
 8: 'TRANSFER FROM HOSPITAL',
 9: 'TRANSFER FROM SKILLED NURSING FACILITY',
 10: 'WALK-IN/SELF REFERRAL',
 11: 'nan'}

In [7]:
tab_features.meta["discharge_location"].get_mapping()

{0: 'ACUTE HOSPITAL',
 1: 'AGAINST ADVICE',
 2: 'ASSISTED LIVING',
 3: 'CHRONIC/LONG TERM ACUTE CARE',
 4: 'DIED',
 5: 'HEALTHCARE FACILITY',
 6: 'HOME',
 7: 'HOME HEALTH CARE',
 8: 'HOSPICE',
 9: 'OTHER FACILITY',
 10: 'PSYCH FACILITY',
 11: 'REHAB',
 12: 'SKILLED NURSING FACILITY',
 13: 'nan'}

In [8]:
tab_features.meta["admission_type"].get_mapping()

{0: 'AMBULATORY OBSERVATION',
 1: 'DIRECT EMER.',
 2: 'DIRECT OBSERVATION',
 3: 'ELECTIVE',
 4: 'EU OBSERVATION',
 5: 'EW EMER.',
 6: 'OBSERVATION ADMIT',
 7: 'SURGICAL SAME DAY ADMISSION',
 8: 'URGENT'}

In [9]:
tab_vectorized = tab_features.vectorize()
tab_vectorized.data.shape

/home/kmckeen/gemini/cyclops/cyclops/processors/feature/feature.py:256: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].astype(int)


(224764, 6)

In [10]:
tab_vectorized.axis_names

['by', 'features']

In [11]:
tab_vectorized.indexes[1]

array(['sex', 'discharge_location', 'admission_location', 'ethnicity',
       'admission_type', 'age'], dtype='<U18')

In [28]:
tab_feature_vals = tab_features.data.sort_values(ENCOUNTER_ID)[SEX].values.astype(float)
tab_feature_vals

array([1., 0., 0., ..., 1., 1., 1.])

In [29]:
tab_vectorized_vals = tab_vectorized.data[:, tab_vectorized.index_maps[1]["sex"]]
tab_vectorized_vals

array([1., 0., 0., ..., 1., 1., 1.])

In [30]:
# WHY ISN'T THIS WORKING??
(tab_feature_vals - tab_vectorized_vals == 0).all()

False

## Temporal

In [ ]:
events = load_dataframe("events.parquet")

# Process
events[EVENT_NAME] = normalize_names(events[EVENT_NAME])
events[EVENT_CATEGORY] = normalize_categories(events[EVENT_CATEGORY])
# events[EVENT_VALUE] = normalize_values(events[EVENT_VALUE])

# Same event names are the same, but mean different things for different categories
# E.g., 'flow' for categories heartware and ecmo
events[EVENT_NAME] = events[EVENT_NAME] + " - " + events[EVENT_CATEGORY]
events.head(5)

In [ ]:
# Keep only the most popular events, including the target
top_events = events[EVENT_NAME].value_counts()[:150].index
top_events = np.append(top_events, TARGET_NAME).unique()
print(top_events)
events = events[events[EVENT_NAME].isin(top_events)]
events

In [ ]:
events.head(5)

In [ ]:
events[events[EVENT_NAME].str.contains("death")].head(5)

In [ ]:
aggregator = Aggregator(
    aggfuncs={EVENT_VALUE: MEAN},
    timestamp_col=EVENT_TIMESTAMP,
    time_by=ENCOUNTER_ID,
    agg_by=[ENCOUNTER_ID, EVENT_NAME],
    timestep_size=8,
    window_duration=24,  # Optional
    # agg_meta_for=EVENT_VALUE,  # Optional
)

In [ ]:
aggregated = tmp_features.aggregate()
aggregated.head(5)

In [ ]:
aggregated.head(100)

In [ ]:
save_dataframe(aggregated, "aggregated.parquet")

In [ ]:
temp_vectorized = aggregator.vectorize(aggregated)
temp_vectorized.data.shape

In [ ]:
temp_vectorized.axis_names

## Normalization

In [ ]:
tab_normalizer = GroupbyNormalizer({EVENT_VALUE: STANDARD}, by=EVENT_NAME)
tab_feature.add_normali

In [ ]:
feat_axis = vectorized.axis_names.index(FEATURES)
temp_normalizer = VectorizedNormalizer(
    feat_axis,
    normalization_method=STANDARD,
)

In [ ]:
# tmp_features.normalize(FEATURES)
# tmp_features.get_data()

In [ ]:
temp_vectorized = np.squeeze(temp_vectorized)
temp_vectorized.shape

In [ ]:
train_data, val_data, test_data = split_data(temp_vectorized, [0.7, 0.2])

In [ ]:
train_data.shape

In [ ]:
val_data.shape

In [ ]:
test_data.shape